<center><h1 style="font-size:60px;"><b>HeartDisease_Data_Analysis</b></h1></center>
<b>This notebook reads in the HeartDisease data and preprocess the data further. It then performs some statistical analysis on the data. Some machine learning algorithm such as Random forest, Deep Neural Network, logistic regression and kNN are also used to model the data.</b>

<h1 style="font-size:60px;"><b>Import packages</b></h1>

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency, shapiro, ttest_ind, ks_2samp, mannwhitneyu
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from dython import nominal
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    auc,
    matthews_corrcoef,
    plot_roc_curve,
    roc_curve,
)
from sklearn.utils import resample
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

<h1 style="font-size:60px;"><b>Read data, clean </b></h1>

In [ ]:
df = pd.read_csv("Heart_disease.csv", index_col=0)

In [ ]:
###Diabetic and Race are categorical variables with more than two levels
expand_df = pd.get_dummies(df, columns=["Diabetic", "Race"])

In [ ]:
expand_df = expand_df.rename(
    columns={
        "Diabetic_No, borderline diabetes": "DiabeticBorderline",
        "Diabetic_Yes (during pregnancy)": "DiabeticYesPregnancy",
        "Race_American Indian/Alaskan Native": "RaceAmericanIndianAlaskanNative",
        "Race_Asian": "RaceAsian",
        "Race_Black": "RaceBlack",
        "Race_Hispanic": "RaceHispanic",
        "Race_Other": "RaceOther",
        "Race_White": "RaceWhite",
    }
)

<h1 style="font-size:60px;"><b>Input</b></h1>

In [ ]:
###this is the feature to be used as y_of_model
###"Stroke" "HeartDisease" "DeadorAlive"
###if y_of_model is "DeadorAlive" declare a subsetting variable
y_of_model = "HeartDisease"
subset_variable = " "  ##"HeartDisease" "Stroke" or ""
if y_of_model == "DeadorAlive":
    df = df.loc[df[subset_variable] == "Yes"]
    expand_df = expand_df.loc[expand_df[subset_variable] == "Yes"]
else:
    print("nothing to subset")

<h1 style="font-size:60px;"><b>Statistical Analysis</b></h1>

In [ ]:
###category_vs_category
##'HeartDisease' removed from the other categorical_features
categorical_features = [
    "HeartDisease",
    "Smoking",
    "Diabetic",
    "AlcoholDrinking",
    "Stroke",
    "DiffWalking",
    "Sex",
    "Race",
    "PhysicalActivity",
    "Asthma",
    "KidneyDisease",
    "SkinCancer",
    "DeadorAlive",
]
categorical_features.remove(y_of_model)
for feature in categorical_features:
    chisqt = pd.crosstab(eval("df." + y_of_model), eval("df." + feature), margins=True)
    value = np.array(
        [
            chisqt.iloc[0][0 : (chisqt.shape[1] - 1)].values,
            chisqt.iloc[1][0 : (chisqt.shape[1] - 1)].values,
        ]
    )
    print(feature + " results:" + str(chi2_contingency(value)[0:3]))

In [ ]:
####category_vs_continuous(and count data)
continous_and_count_data = ["BMI", "SleepTime", "PhysicalHealth", "MentalHealth"]
for continuous_or_count in continous_and_count_data:
    NO_y_of_model = df.loc[df[y_of_model] == "No", continuous_or_count]
    YES_y_of_model = df.loc[df[y_of_model] == "Yes", continuous_or_count]
    if (
        shapiro(np.array(eval("df." + continuous_or_count)))[1] <= 0.05
    ) == True:  # normality check
        print(
            "non-parametric results for "
            + continuous_or_count
            + " "
            + str(ks_2samp(NO_y_of_model, YES_y_of_model)[1])
        )
    else:
        print(
            "parametric results for "
            + continuous_or_count
            + " "
            + str(ttest_ind(NO_y_of_model, YES_y_of_model, equal_var=False)[1])
        )

In [ ]:
####categorical_vs_ordinal
ordinal_data = ["AgeCategory", "GenHealth"]
for ordinal_feature in ordinal_data:
    NO_y_of_model = df.loc[df[y_of_model] == "No", ordinal_feature]
    YES_y_of_model = df.loc[df[y_of_model] == "Yes", ordinal_feature]
    res = mannwhitneyu(x=NO_y_of_model, y=YES_y_of_model, alternative="two-sided")[1]
    print(ordinal_feature + " result: " + str(res))

<h1 style="font-size:60px;"><b>Preprocessing for machine learning</b></h1>

In [ ]:
if y_of_model != "DeadorAlive":
    expand_df = expand_df.drop(columns=["DeadorAlive"])
else:
    pass

In [ ]:
categorical_features = [
    "HeartDisease",
    "Smoking",
    "AlcoholDrinking",
    "Stroke",
    "DiffWalking",
    "PhysicalActivity",
    "Asthma",
    "KidneyDisease",
    "SkinCancer",
]
try:
    categorical_features.remove(y_of_model)
except:
    pass
try:
    categorical_features.remove(subset_variable)
except:
    pass
for feature in categorical_features:
    expand_df[feature] = expand_df[feature].replace(["No", "Yes"], [0, 1])

expand_df["Sex"] = expand_df["Sex"].replace(["Male", "Female"], [0, 1])

In [ ]:
####downsampling:
expand_df_yes = expand_df[expand_df[y_of_model] == "Yes"]
expand_df_no = expand_df[expand_df[y_of_model] == "No"]
expand_df_no_downsample = resample(
    expand_df_no, replace=True, n_samples=len(expand_df_yes), random_state=42
)
expand_df_downsampled = pd.concat([expand_df_no_downsample, expand_df_yes])
try:
    expand_df_downsampled = expand_df_downsampled.drop(subset_variable, axis=1)
except:
    pass

In [ ]:
####standardscaling of continuous and count variables
continous_and_count_data_and_ordinal = [
    "BMI",
    "SleepTime",
    "PhysicalHealth",
    "MentalHealth",
    "AgeCategory",
    "GenHealth",
]
for candcount in continous_and_count_data_and_ordinal:
    scaler = StandardScaler()
    scaler.fit(expand_df_downsampled[[candcount]])
    expand_df_downsampled[[candcount]] = scaler.transform(
        expand_df_downsampled[[candcount]]
    )

In [ ]:
expand_df = expand_df_downsampled

In [ ]:
expand_df[y_of_model] = expand_df[y_of_model].replace(["No", "Yes"], [0, 1])
y = expand_df[[y_of_model]]
X = expand_df.drop(y_of_model, axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.09, random_state=0, shuffle=True
)

<h1 style="font-size:60px;"><b>Random Forest</b></h1>

In [ ]:
### print model and matthews_corrcoef scores (-1, 0, +1)
name_of_model = "RandomForestClassifier"
res_list = []
print("max_features-max_depth-n_estimators-MCC-accuracy")
for x in [10, 15, 20]:
    for j in [20, 25, 30, 35]:
        for w in [20, 40, 80, 100]:
            clf = RandomForestClassifier(
                n_estimators=w,
                criterion="entropy",
                max_depth=j,
                max_features=x,
                n_jobs=4,
            )
            clf = clf.fit(X_train, eval("y_train." + y_of_model + ".ravel()"))
            y_pred = clf.predict(X_test)
            res = (
                str(x)
                + "-"
                + str(j)
                + "-"
                + str(w)
                + "-"
                + str(matthews_corrcoef(y_test, y_pred))
                + "-"
                + str(accuracy_score(y_test, y_pred))
            )
            print(res)
            res_list.append(res)

In [ ]:
feature_imp = pd.Series(clf.feature_importances_, index=X_train.columns).sort_values(
    ascending=False
)

In [ ]:
sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel("Feature Importance Score")
plt.ylabel("Features")
plt.title(y_of_model + " risk")
plt.show()

In [ ]:
y_pred_proba = clf.predict_proba(X_test)[::, 1]
fpr, tpr, _ = roc_curve(eval("y_test." + y_of_model + ".ravel()"), y_pred_proba)
auc_result = str(round(auc(fpr, tpr), 2))
plt.fill_between(
    fpr,
    tpr,
    color="green",
    alpha=0.5,
    label="{} ({})".format(name_of_model, auc_result),
)
plt.plot(fpr, tpr)
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.title("ROC of {} ({})".format(name_of_model, y_of_model))
plt.legend(loc="lower right")
plt.show()

<h1 style="font-size:60px;"><b>Deep Neural Network</b></h1>

In [ ]:
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(12, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(12, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(12, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(12, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(
    Dense(len(eval("y_train." + y_of_model + ".unique()")) - 1, activation="sigmoid")
)

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
es = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=13,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

In [ ]:
history = model.fit(
    X,
    eval("y." + y_of_model + ".ravel()"),
    epochs=40,
    callbacks=[es],
    validation_split=0.09,
    batch_size=10,
).history

In [ ]:
plt.plot(history["accuracy"], label="training")
plt.plot(history["val_accuracy"], label="validation")
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.legend(loc="upper left")

<h1 style="font-size:60px;"><b>Logistic Regression</b></h1>

In [ ]:
X_train_log, X_test_log = train_test_split(
    expand_df_downsampled, test_size=0.09, random_state=0, shuffle=True
)

In [ ]:
independent_variables = list(X_train_log.columns)
independent_variables.remove(y_of_model)
formular = y_of_model + " ~ "
for feature in independent_variables:
    formular = formular + " + " + feature
formular = formular.replace("+", "", 1)

In [ ]:
model = smf.logit(formular, data=X_train_log)
model = model.fit_regularized(start_params=None, method="l1", alpha=0)
print(model.summary())

In [ ]:
print(model.params)

In [ ]:
model_odds = pd.DataFrame(np.exp(model.params), columns=["OR"])
model_odds["z-value"] = model.pvalues
model_odds[["2.5%", "97.5%"]] = np.exp(model.conf_int())

print(model_odds)

In [ ]:
odd_ratio = model_odds[model_odds["z-value"] <= 0.99].sort_values("OR", ascending=False)

In [ ]:
sns.barplot(x=odd_ratio["OR"], y=odd_ratio.index)
plt.xlabel("OddRatio")
plt.ylabel("Features")
plt.axvline(1)
plt.title(y_of_model + " OddRatio")
plt.show()

In [ ]:
pred = model.predict(exog=X_test_log[independent_variables])
accuracy_score(y_true=list(X_test_log[y_of_model]), y_pred=list(round(pred)))

<h1 style="font-size:60px;"><b>KNN</b></h1>

In [ ]:
res_list = []
for k in range(2, 50):
    clf = KNeighborsClassifier(n_neighbors=k, p=1)
    clf = clf.fit(X_train, eval("y_train." + y_of_model + ".ravel()"))
    y_pred = clf.predict(X_test)
    res = str(k) + "-" + str(matthews_corrcoef(y_test, y_pred))
    print(res)
    res_list.append(res)